In [94]:
import gist

from PIL import Image
import imageio
from IPython.display import display, clear_output

import sys
sys.path.append("../Library/")
import image_download as imd
import image_manipulation as ima

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import os
load_dotenv("../.env")
import csv

import numpy as np
import db_connection as dbcon

#%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Set Parameters

In [95]:
# parameters
GMAPS_API_KEY = os.getenv('GMAPS_API_KEY')
IMG_FOLDER = os.getenv('MFP_IMG_FOLDER')

# connect to db
db = dbcon.connect("../credentials/mlab_db.txt","mfp")
images_lib_col = db["images_lib"]

DB Credentials from file
DB connected successfully!!!


### Load filename to label from query to DB

In [96]:
query = {"name": "US1"}
fnames = ima.get_metadata_filenames(images_lib_col, query)
fnames

Existing images metadata: 20


['US1_45.21160965529817_-93.54554509042913_13_640.png',
 'US1_45.21160965529817_-93.54554509042913_14_640.png',
 'US1_45.21160965529817_-93.54554509042913_15_640.png',
 'US1_45.21160965529817_-93.54554509042913_16_640.png',
 'US1_52.20482183860124_-92.48891258998856_13_640.png',
 'US1_52.20482183860124_-92.48891258998856_14_640.png',
 'US1_52.20482183860124_-92.48891258998856_15_640.png',
 'US1_52.20482183860124_-92.48891258998856_16_640.png',
 'US1_33.01060691483008_-118.04101400786547_13_640.png',
 'US1_33.01060691483008_-118.04101400786547_14_640.png',
 'US1_33.01060691483008_-118.04101400786547_15_640.png',
 'US1_33.01060691483008_-118.04101400786547_16_640.png',
 'US1_50.53186589235239_-108.35386180596005_13_640.png',
 'US1_50.53186589235239_-108.35386180596005_14_640.png',
 'US1_50.53186589235239_-108.35386180596005_15_640.png',
 'US1_50.53186589235239_-108.35386180596005_16_640.png',
 'US1_45.14093597874625_-101.4861640355236_13_640.png',
 'US1_45.14093597874625_-101.48616403552

### Get image info of all the files of area name provided above (filenames, images, np.array)

In [97]:
images_info = ima.load_images_from_gdrive(fnames, IMG_FOLDER)

### Start labeling app and save csv

In [99]:
## !!! THIS WILL OPEN AN INTERACTIVE WINDOW FOR LABELLING THE IMAGES!!!
## FIRST PUT IN YOUR NAME AND THEN GO AHEAD WITH LABELLING
filename_upload_to_db = 'temp2.csv'
images_info = ima.add_labels_and_save_csv(images_info, IMG_FOLDER, filename_upload_to_db)

### Label images in Database

In [100]:
dbcon.write_labels_from_csv_to_db(images_lib_col, IMG_FOLDER, filename_upload_to_db)

loading labels label_multi_er and label_binary_er to db ...
20 labels added to db!


### Check binary labels

In [101]:
label_name = 'label_binary_er'
query = {label_name: '0'}

doc = images_lib_col.find(query)
for x in doc:
    print(x['filename'], "has label", x[label_name])

US1_52.20482183860124_-92.48891258998856_13_640.png has label 0
US1_52.20482183860124_-92.48891258998856_14_640.png has label 0
US1_52.20482183860124_-92.48891258998856_15_640.png has label 0
US1_52.20482183860124_-92.48891258998856_16_640.png has label 0
US1_33.01060691483008_-118.04101400786547_13_640.png has label 0


In [102]:
label_name = 'label_binary_er'
query = {label_name: '1'}

doc = images_lib_col.find(query)
for x in doc:
    print(x['filename'], "has label", x[label_name])

US1_45.21160965529817_-93.54554509042913_13_640.png has label 1
US1_45.21160965529817_-93.54554509042913_14_640.png has label 1
US1_45.21160965529817_-93.54554509042913_15_640.png has label 1
US1_45.21160965529817_-93.54554509042913_16_640.png has label 1
US1_33.01060691483008_-118.04101400786547_14_640.png has label 1
US1_33.01060691483008_-118.04101400786547_15_640.png has label 1
US1_33.01060691483008_-118.04101400786547_16_640.png has label 1
US1_50.53186589235239_-108.35386180596005_13_640.png has label 1
US1_50.53186589235239_-108.35386180596005_14_640.png has label 1
US1_50.53186589235239_-108.35386180596005_15_640.png has label 1
US1_50.53186589235239_-108.35386180596005_16_640.png has label 1
US1_45.14093597874625_-101.4861640355236_13_640.png has label 1
US1_45.14093597874625_-101.4861640355236_14_640.png has label 1
US1_45.14093597874625_-101.4861640355236_15_640.png has label 1
US1_45.14093597874625_-101.4861640355236_16_640.png has label 1
